In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import cv2
import os


In [ ]:
from keras.models import load_model
from keras.optimizers import SGD

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

model = load_model("/content/drive/MyDrive/Project Data/flow_gated_fighting_bs_8.h5")



In [ ]:
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
def add_optical_flow(video):
    """Calculate dense optical flow of input video
    Args:
        video: the input video with shape of [frames,height,width,channel]. dtype=np.array
    Returns:
        flows_x: the optical flow at x-axis, with the shape of [frames,height,width,channel]
        flows_y: the optical flow at y-axis, with the shape of [frames,height,width,channel]
    """
    # initialize the list of optical flows
#     print(len(video))
    gray_video = []
    for i in range(len(video)):
        img = cv2.cvtColor(video[i], cv2.COLOR_RGB2GRAY)
        gray_video.append(np.reshape(img,(224,224,1)))

    flows = []
    for i in range(len(video)-1):
        # calculate optical flow between each pair of frames
        flow = cv2.calcOpticalFlowFarneback(gray_video[i], gray_video[i+1], None, 0.5, 3, 15, 3, 5, 1.2, cv2.OPTFLOW_FARNEBACK_GAUSSIAN)
        # subtract the mean in order to eliminate the movement of camera
        flow[..., 0] -= np.mean(flow[..., 0])
        flow[..., 1] -= np.mean(flow[..., 1])
        # normalize each component in optical flow
        flow[..., 0] = cv2.normalize(flow[..., 0],None,0,255,cv2.NORM_MINMAX)
        flow[..., 1] = cv2.normalize(flow[..., 1],None,0,255,cv2.NORM_MINMAX)
        # Add into list 
        flows.append(flow)
        
    # Padding the last frame as empty array
    flows.append(np.zeros((224,224,2)))
    
    
    
    result = np.zeros((len(flows),224,224,5))
    result[...,:3] = video
    result[...,3:] = flows
    return np.array(result, dtype=np.float32)

In [ ]:
def video2numpy(cap):
    frameCount = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frameWidth = 224
    frameHeight = 224

    buf = np.empty((frameCount, frameHeight, frameWidth, 3), np.dtype('uint8'))

    fc = 0
    ret = True

    while (fc < frameCount  and ret):
        ret,frame = cap.read()
        buf[fc]=cv2.resize(frame,(224,224))
        fc += 1
#     print(buf.shape)
      
    return(buf)

In [ ]:
def pred(video_with_OF):
    result_list=[]
    for i in range(len(video_with_OF)//64):
        segment=np.empty((64,224,224,5))
        for i in range(64):
            segment[i]=video_with_OF[i]
        result_list.append(model.predict(segment.reshape(1,64,224,224,5)))
        video_with_OF=video_with_OF[64:,:,:,:]
    return(result_list)

In [ ]:
def show(video,result_list):
    length=len(video)
    font = cv2.FONT_HERSHEY_SIMPLEX
    k=0
    for l in range(length):
        
        if l%64==0:
            if result_list[k] [0] [0] > 0.7:
                text = "anomaly"
                color  = (0,0,255)
            else:
                text = "normal"
                color  = (0,255,0)
        
        frame=video[l]
        cv2.putText(frame,text,(50, 50),font, 1,(255, 0, 0),2,cv2.LINE_4)
        cv2.imshow('video',frame)
        key=cv2.waitKey(0)
        if key & 0xFF == ord('q'): 
            break

In [ ]:
a_folder = "/content/drive/MyDrive/Project Data/Fighting"
a_videos=os.listdir(a_folder)
a_videos

['Fighting003_x264.mp4',
 'Fighting006_x264.mp4',
 'Fighting010_x264.mp4',
 'Fighting011_x264.mp4',
 'Fighting012_x264.mp4',
 'Fighting013_x264.mp4',
 'Normal_Videos_034_x264.mp4',
 'Normal_Videos_041_x264.mp4',
 'Normal_Videos_042_x264.mp4',
 'Normal_Videos_050_x264.mp4',
 'Normal_Videos_048_x264.mp4',
 'Anomaly_27_.mp4']

In [ ]:
video_name=a_videos[-1]
video_name

'Anomaly_27_.mp4'

In [ ]:
cap=cv2.VideoCapture(os.path.join(a_folder,video_name))


In [ ]:
video=video2numpy(cap)


In [ ]:
video_with_OF=add_optical_flow(video)


In [ ]:
result_list=pred(video_with_OF)


In [ ]:
result_list

[array([[9.9999917e-01, 7.9114199e-07]], dtype=float32),
 array([[1.0000000e+00, 9.2169226e-14]], dtype=float32),
 array([[1.0000000e+00, 1.4099862e-12]], dtype=float32)]

In [ ]:
#show(video,result_list)